In [1]:
import os
import librosa
import numpy as np
import csv
import torch
import random
import copy

n_mfcc = 39
csv_file_save = './csv/features.csv'
calculate_csv = None

if not os.path.exists('./csv'):
    os.makedirs('./csv')

if os.path.exists(csv_file_save):
    print ("File " + csv_file_save + " exists")
    print ("Features won't be recalculated")
    calculate_csv = False
else:
    print ("File " + csv_file_save + " doesn't exists")
    print ("Features will be calculated")
    calculate_csv = True

File ./csv/features.csv doesn't exists
Features will be calculated


In [2]:
# extract info speakers 

file_speaker = "./LibriSpeech/SPEAKERS.TXT"
speakers = []

f = open(file_speaker, "r")
for line in f:
    if line[0] == ";":
        continue
    else:
        speakers.append( str.rstrip(line) ) # rstrip to remove \n at the end of the string

In [3]:
speakers[:10]

['14   | F | train-clean-360  | 25.03 | Kristin LeMoine',
 '16   | F | train-clean-360  | 25.11 | Alys AtteWater',
 '17   | M | train-clean-360  | 25.04 | Gord Mackenzie',
 '19   | F | train-clean-100  | 25.19 | Kara Shallenberg',
 '20   | F | train-other-500  | 30.07 | Gesine',
 '22   | F | train-clean-360  | 25.14 | Michelle Crandall',
 '23   | F | train-clean-360  | 25.23 | Anita Roy Dobbs',
 '25   | M | train-other-500  | 30.16 | John Gonzalez',
 '26   | M | train-clean-100  | 25.08 | Denny Sayers',
 '27   | M | train-clean-100  | 20.14 | Sean McKinley']

In [4]:
# extract speakers gender 
# store in dictionary and in list to choose train and test speakers

dict_speakers = {}
list_speakers = []
for speaker in speakers:
    speaker_split = speaker.split()
    speaker_split = [word for word in speaker_split if word != "|"]
    
    # indexes = 0 : id, 1 : gender, 2 : dataset
    if speaker_split[2] == "dev-clean":
        dict_speakers[speaker_split[0]] = speaker_split[1]
        list_speakers.append(speaker_split[0])
    else:
        continue

In [5]:
dict_speakers

{'84': 'F',
 '174': 'M',
 '251': 'M',
 '422': 'M',
 '652': 'M',
 '777': 'M',
 '1272': 'M',
 '1462': 'F',
 '1673': 'F',
 '1919': 'F',
 '1988': 'F',
 '1993': 'F',
 '2035': 'F',
 '2078': 'M',
 '2086': 'M',
 '2277': 'F',
 '2412': 'F',
 '2428': 'M',
 '2803': 'M',
 '2902': 'M',
 '3000': 'M',
 '3081': 'F',
 '3170': 'M',
 '3536': 'F',
 '3576': 'F',
 '3752': 'M',
 '3853': 'F',
 '5338': 'F',
 '5536': 'M',
 '5694': 'M',
 '5895': 'F',
 '6241': 'M',
 '6295': 'M',
 '6313': 'F',
 '6319': 'F',
 '6345': 'F',
 '7850': 'F',
 '7976': 'M',
 '8297': 'M',
 '8842': 'F'}

In [6]:
# get audio files info : name and path 

if calculate_csv:
    audio_files = []

    root = "./"
    path = os.path.join(root, "LibriSpeech")

    for (dirpath, dirnames, filenames) in os.walk(path):
        for file in filenames:
            if file[-5:] == ".flac":
                audio_files.append({"dirpath": dirpath, "filename": file})

In [7]:
if calculate_csv:
    print(len(audio_files))

2703


In [8]:
if calculate_csv:
    print(audio_files[:10])

[{'dirpath': './LibriSpeech\\dev-clean\\1272\\128104', 'filename': '1272-128104-0000.flac'}, {'dirpath': './LibriSpeech\\dev-clean\\1272\\128104', 'filename': '1272-128104-0001.flac'}, {'dirpath': './LibriSpeech\\dev-clean\\1272\\128104', 'filename': '1272-128104-0002.flac'}, {'dirpath': './LibriSpeech\\dev-clean\\1272\\128104', 'filename': '1272-128104-0003.flac'}, {'dirpath': './LibriSpeech\\dev-clean\\1272\\128104', 'filename': '1272-128104-0004.flac'}, {'dirpath': './LibriSpeech\\dev-clean\\1272\\128104', 'filename': '1272-128104-0005.flac'}, {'dirpath': './LibriSpeech\\dev-clean\\1272\\128104', 'filename': '1272-128104-0006.flac'}, {'dirpath': './LibriSpeech\\dev-clean\\1272\\128104', 'filename': '1272-128104-0007.flac'}, {'dirpath': './LibriSpeech\\dev-clean\\1272\\128104', 'filename': '1272-128104-0008.flac'}, {'dirpath': './LibriSpeech\\dev-clean\\1272\\128104', 'filename': '1272-128104-0009.flac'}]


In [9]:
# get labels genders (target) for each file 

if calculate_csv:
    gender_labels = []

    for audio_file in audio_files:
        file = audio_file["filename"]
        index = 0
        for char in file:
            if char != "-":
                index += 1
            else:
                break
        id_speaker = file[:index]
        audio_file["id_speaker"] = id_speaker

        gender_labels.append(dict_speakers[id_speaker])

In [10]:
if calculate_csv:
    print(len(gender_labels))

2703


In [11]:
# extract mfcc features and make mean for each feature

if calculate_csv:

    input_features = []

    counter = 0

    for audio_file in audio_files:

        counter += 1
        if counter % 100 == 0:
            print("Extracting features file " + str(counter) + "/" + str(len(audio_files)))

        directory = audio_file["dirpath"]
        filename = audio_file["filename"]

        y, sr = librosa.load(directory + "/" + filename)
        hop_length = 512
        mfccs = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=n_mfcc)
        mfccs_processed = np.mean(mfccs.T,axis=0)
        input_features.append(mfccs_processed)

    print("End of extracting features")

Extracting features file 100/2703
Extracting features file 200/2703
Extracting features file 300/2703
Extracting features file 400/2703
Extracting features file 500/2703
Extracting features file 600/2703
Extracting features file 700/2703
Extracting features file 800/2703
Extracting features file 900/2703
Extracting features file 1000/2703
Extracting features file 1100/2703
Extracting features file 1200/2703
Extracting features file 1300/2703
Extracting features file 1400/2703
Extracting features file 1500/2703
Extracting features file 1600/2703
Extracting features file 1700/2703
Extracting features file 1800/2703
Extracting features file 1900/2703
Extracting features file 2000/2703
Extracting features file 2100/2703
Extracting features file 2200/2703
Extracting features file 2300/2703
Extracting features file 2400/2703
Extracting features file 2500/2703
Extracting features file 2600/2703
Extracting features file 2700/2703
End of extracting features


In [12]:
if calculate_csv:
    print(len(input_features))

2703


In [13]:
if calculate_csv:
    print(len(input_features[0]))

39


In [14]:
# save features on csv with additional info and labels

if calculate_csv:

    with open(csv_file_save, 'w', newline='') as csvfile:
        feat_writer = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)

        index = 0

        row_to_write = ['label', 'label_value', 'id_speaker']
        for f in range(n_mfcc):
            row_to_write.append('f' + str(f))

        feat_writer.writerow(row_to_write)

        for input_feature in input_features:

            class_label = gender_labels[index]
            if class_label == "M":
                class_label = 0
            else:
                class_label = 1

            row_to_write = [class_label, gender_labels[index], audio_files[index]["id_speaker"]]

            index += 1

            if index % 500 == 0:
                print("Writing row " + str(index) + "/" + str(len(audio_files)))
            for features in input_feature:
                row_to_write.append(features)

            feat_writer.writerow(row_to_write)
    print("End of writing csv")

Writing row 500/2703
Writing row 1000/2703
Writing row 1500/2703
Writing row 2000/2703
Writing row 2500/2703
End of writing csv


In [15]:
# choose speakers to put in each dataset

random.shuffle(list_speakers)

speakers_train = []
speakers_val   = []
speakers_test  = []
count_males   = 0
count_females = 0 
n_speakers = len(list_speakers)
n_speakers_for_gender_train = (n_speakers / 2) * 0.7
n_speakers_for_gender_train_val = (n_speakers / 2) * 0.85

for speaker in list_speakers:
    gender = dict_speakers[speaker]
    if gender == 'M':
        if count_males < n_speakers_for_gender_train: 
            speakers_train.append(int(speaker))
        elif count_males < n_speakers_for_gender_train_val: 
            speakers_val.append(int(speaker))
        else:
            speakers_test.append(int(speaker))
        count_males += 1
    elif gender == 'F':
        if count_females < n_speakers_for_gender_train: 
            speakers_train.append(int(speaker))
        elif count_females < n_speakers_for_gender_train_val: 
            speakers_val.append(int(speaker))
        else:
            speakers_test.append(int(speaker))
        count_females += 1
    
print(speakers_train)
print(speakers_val)
print(speakers_test)

[1272, 1673, 3752, 7850, 3000, 8842, 2078, 84, 2277, 3081, 422, 5338, 1462, 7976, 251, 6319, 652, 3170, 6313, 2428, 6345, 777, 8297, 6295, 5536, 3853, 3576, 2412]
[6241, 2803, 2902, 3536, 1988, 2035]
[174, 2086, 1993, 5895, 5694, 1919]


In [16]:
# read csv to get inputs and targets

my_data = np.genfromtxt(csv_file_save, delimiter=',', skip_header=1)

labels_train = []
data_train   = []
labels_val   = []
data_val     = []
labels_test  = []
data_test    = []
labels_train_svm = []
labels_val_svm = []
labels_test_svm  = []

for row in my_data:
    if row[2] in speakers_train:
        labels_train.append([row[0]])
        data_train.append(row[3:])
        if row[0] == 1:
            labels_train_svm.append([1])
        else:
            labels_train_svm.append([-1])
    elif row[2] in speakers_val:
        labels_val.append([row[0]])
        data_val.append(row[3:])
        if row[0] == 1:
            labels_val_svm.append([1])
        else:
            labels_val_svm.append([-1])
    else:
        labels_test.append([row[0]])
        data_test.append(row[3:])
        if row[0] == 1:
            labels_test_svm.append([1])
        else:
            labels_test_svm.append([-1])

data_train = list(zip(labels_train, data_train, labels_train_svm))
random.shuffle(data_train)
labels_train, data_train, labels_train_svm = zip(*data_train)

data_val = list(zip(labels_val, data_val, labels_val_svm))
random.shuffle(data_val)
labels_val, data_val, labels_val_svm = zip(*data_val)

data_test = list(zip(labels_test, data_test, labels_test_svm))
random.shuffle(data_test)
labels_test, data_test, labels_test_svm = zip(*data_test)

labels_train      = np.array(labels_train)
data_train        = np.array(data_train)
labels_val        = np.array(labels_val)
data_val          = np.array(data_val)
labels_test       = np.array(labels_test)
data_test         = np.array(data_test)
labels_train_svm  = np.array(labels_train_svm)
labels_val_svm    = np.array(labels_val_svm)
labels_test_svm   = np.array(labels_test_svm)

print(labels_train.shape)
print(data_train.shape)
print(labels_val.shape)
print(data_val.shape)
print(labels_test.shape)
print(data_test.shape)
print(labels_train_svm.shape)
print(labels_val_svm.shape)
print(labels_test_svm.shape)

(1915, 1)
(1915, 39)
(389, 1)
(389, 39)
(399, 1)
(399, 39)
(1915, 1)
(389, 1)
(399, 1)


In [17]:
# cast data to tensor

data_train_tensor   = torch.from_numpy(data_train).float()
labels_train_tensor = torch.from_numpy(labels_train).float()
data_val_tensor   = torch.from_numpy(data_val).float()
labels_val_tensor = torch.from_numpy(labels_val).float()
data_test_tensor    = torch.from_numpy(data_test).float()
labels_test_tensor  = torch.from_numpy(labels_test).float()

print(data_train_tensor.shape, labels_train_tensor.shape)
print(data_val_tensor.shape, labels_val_tensor.shape)
print(data_test_tensor.shape, labels_test_tensor.shape)

labels_train_svm_tensor = torch.from_numpy(labels_train_svm).float()
labels_val_svm_tensor   = torch.from_numpy(labels_val_svm).float()
labels_test_svm_tensor  = torch.from_numpy(labels_test_svm).float()

torch.Size([1915, 39]) torch.Size([1915, 1])
torch.Size([389, 39]) torch.Size([389, 1])
torch.Size([399, 39]) torch.Size([399, 1])


In [18]:
# kNN MODEL WITH RESULTS

label_pred_val = []

for val_data in data_val_tensor:
    dist = torch.norm(data_train_tensor - val_data, dim=1, p=None)
    knn  = dist.topk(3, largest=False)
    
    result = 0
    for index in knn.indices:
        result += int(labels_train_tensor[index])
    result = (result / 3)
    
    label_pred_val.append([result])

label_pred_val = torch.as_tensor(label_pred_val)

# Accuracy test 
output_val  = (label_pred_val > 0.5).float()
correct_val = (output_val == labels_val_tensor).float().sum()

print("Accuracy_Val: {:.3f}".format(correct_val/labels_val_tensor.shape[0]))

label_pred_test = []

for test_data in data_test_tensor:
    dist = torch.norm(data_train_tensor - test_data, dim=1, p=None)
    knn = dist.topk(3, largest=False)
    
    result = 0
    for index in knn.indices:
        result += int(labels_train_tensor[index])
    result = (result / 3)
    
    label_pred_test.append([result])

label_pred_test = torch.as_tensor(label_pred_test)

# Accuracy test 
output_test  = (label_pred_test > 0.5).float()
correct_test = (output_test == labels_test_tensor).float().sum()

print("Accuracy_Test: {:.3f}".format(correct_test/labels_test_tensor.shape[0]))

Accuracy_Val: 0.728
Accuracy_Test: 0.779


In [19]:
# NN MODEL WITH RESULTS

hidden_neurons = 32
n_outputs = 1
num_epochs = 10000

model = torch.nn.Sequential(
    torch.nn.Linear(n_mfcc, hidden_neurons),
    torch.nn.ReLU(),
    torch.nn.Dropout(p = 0.1),
    torch.nn.Linear(hidden_neurons, hidden_neurons),
    torch.nn.ReLU(),
    torch.nn.Dropout(p = 0.1),
    torch.nn.Linear(hidden_neurons, n_outputs),
    torch.nn.Sigmoid()
)

criterion = torch.nn.BCELoss(reduction='sum')

learning_rate = 5e-5
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

minimum = 0
best_model = None

no_improve = 0
early_stopping_steps = 49

for epoch in range(num_epochs):
    y_pred = model(data_train_tensor)
        
    loss = criterion(y_pred, labels_train_tensor)
    
    if epoch % 15 == 14:
        
        y_pred_val = model(data_val_tensor)
        loss_val = criterion(y_pred_val, labels_val_tensor)
        
        #Accuracy 
        output       = (y_pred > 0.5).float()
        correct      = (output == labels_train_tensor).float().sum()
        output_val   = (y_pred_val > 0.5).float()
        correct_val  = (output_val == labels_val_tensor).float().sum()
        accuracy_val = correct_val/labels_val_tensor.shape[0]
        
        if accuracy_val > minimum:
            minimum = accuracy_val
            torch.save({'state_dict':model.state_dict(), 'optimizer': optimizer.state_dict()}, 'model.pth.tar')          
            no_improve = 0
        else:
            no_improve += 1
        
        print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}, Loss_Val: {:.3f}, Accuracy_Val: {:.3f}".format(epoch+1,
                        num_epochs, loss, correct/labels_train_tensor.shape[0], loss_val, correct_val/labels_val_tensor.shape[0]))  

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # early stopping
    if no_improve > early_stopping_steps:
        break

checkpoint = torch.load('model.pth.tar')         

model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])

model.eval()

y_pred_val = model(data_val_tensor)
loss_val = criterion(y_pred_val, labels_val_tensor)
output_val  = (y_pred_val > 0.5).float()
correct_val = (output_val == labels_val_tensor).float().sum()
        
print("Final Model, Loss_Val: {:.3f}, Accuracy_Val: {:.3f}".format(loss_val, correct_val/labels_val_tensor.shape[0])) 

y_pred_test = model(data_test_tensor)
loss_test = criterion(y_pred_test, labels_test_tensor)
output_test  = (y_pred_test > 0.5).float()
correct_test = (output_test == labels_test_tensor).float().sum()
        
print("Final Model, Loss_Test: {:.3f}, Accuracy_Test: {:.3f}".format(loss_test, correct_test/labels_test_tensor.shape[0])) 

Epoch 15/10000, Loss: 2087.148, Accuracy: 0.486, Loss_Val: 457.202, Accuracy_Val: 0.483
Epoch 30/10000, Loss: 2012.589, Accuracy: 0.496, Loss_Val: 446.083, Accuracy_Val: 0.473
Epoch 45/10000, Loss: 1897.559, Accuracy: 0.514, Loss_Val: 377.956, Accuracy_Val: 0.519
Epoch 60/10000, Loss: 1754.988, Accuracy: 0.539, Loss_Val: 365.458, Accuracy_Val: 0.530
Epoch 75/10000, Loss: 1732.917, Accuracy: 0.551, Loss_Val: 386.679, Accuracy_Val: 0.517
Epoch 90/10000, Loss: 1645.777, Accuracy: 0.549, Loss_Val: 344.579, Accuracy_Val: 0.560
Epoch 105/10000, Loss: 1578.652, Accuracy: 0.573, Loss_Val: 345.134, Accuracy_Val: 0.586
Epoch 120/10000, Loss: 1595.119, Accuracy: 0.586, Loss_Val: 320.953, Accuracy_Val: 0.586
Epoch 135/10000, Loss: 1628.090, Accuracy: 0.584, Loss_Val: 313.785, Accuracy_Val: 0.594
Epoch 150/10000, Loss: 1416.328, Accuracy: 0.628, Loss_Val: 323.534, Accuracy_Val: 0.571
Epoch 165/10000, Loss: 1496.648, Accuracy: 0.610, Loss_Val: 313.767, Accuracy_Val: 0.568
Epoch 180/10000, Loss: 1399

Epoch 1410/10000, Loss: 336.229, Accuracy: 0.931, Loss_Val: 165.915, Accuracy_Val: 0.789
Epoch 1425/10000, Loss: 337.004, Accuracy: 0.931, Loss_Val: 182.712, Accuracy_Val: 0.766
Epoch 1440/10000, Loss: 323.996, Accuracy: 0.927, Loss_Val: 170.251, Accuracy_Val: 0.792
Epoch 1455/10000, Loss: 327.897, Accuracy: 0.933, Loss_Val: 152.169, Accuracy_Val: 0.820
Epoch 1470/10000, Loss: 312.227, Accuracy: 0.936, Loss_Val: 183.019, Accuracy_Val: 0.766
Epoch 1485/10000, Loss: 331.428, Accuracy: 0.929, Loss_Val: 171.895, Accuracy_Val: 0.802
Epoch 1500/10000, Loss: 336.026, Accuracy: 0.928, Loss_Val: 168.025, Accuracy_Val: 0.802
Epoch 1515/10000, Loss: 304.593, Accuracy: 0.935, Loss_Val: 173.742, Accuracy_Val: 0.769
Epoch 1530/10000, Loss: 296.014, Accuracy: 0.943, Loss_Val: 159.895, Accuracy_Val: 0.815
Epoch 1545/10000, Loss: 303.215, Accuracy: 0.939, Loss_Val: 171.352, Accuracy_Val: 0.774
Epoch 1560/10000, Loss: 291.925, Accuracy: 0.939, Loss_Val: 174.416, Accuracy_Val: 0.779
Epoch 1575/10000, Los

Epoch 2835/10000, Loss: 109.979, Accuracy: 0.979, Loss_Val: 174.741, Accuracy_Val: 0.820
Epoch 2850/10000, Loss: 94.897, Accuracy: 0.984, Loss_Val: 172.293, Accuracy_Val: 0.815
Epoch 2865/10000, Loss: 117.475, Accuracy: 0.982, Loss_Val: 165.519, Accuracy_Val: 0.815
Epoch 2880/10000, Loss: 94.756, Accuracy: 0.983, Loss_Val: 174.804, Accuracy_Val: 0.805
Epoch 2895/10000, Loss: 99.392, Accuracy: 0.982, Loss_Val: 185.027, Accuracy_Val: 0.781
Epoch 2910/10000, Loss: 96.954, Accuracy: 0.980, Loss_Val: 172.744, Accuracy_Val: 0.799
Epoch 2925/10000, Loss: 92.955, Accuracy: 0.985, Loss_Val: 171.468, Accuracy_Val: 0.794
Epoch 2940/10000, Loss: 91.654, Accuracy: 0.984, Loss_Val: 199.563, Accuracy_Val: 0.792
Epoch 2955/10000, Loss: 101.627, Accuracy: 0.979, Loss_Val: 166.387, Accuracy_Val: 0.828
Epoch 2970/10000, Loss: 95.430, Accuracy: 0.984, Loss_Val: 167.955, Accuracy_Val: 0.805
Epoch 2985/10000, Loss: 87.946, Accuracy: 0.987, Loss_Val: 184.485, Accuracy_Val: 0.802
Epoch 3000/10000, Loss: 82.24

In [20]:
# SVM MODEL WITH RESULTS

dim = 39
w = torch.autograd.Variable(torch.rand(dim), requires_grad=True)
b = torch.autograd.Variable(torch.rand(1),   requires_grad=True)

step_size = 3e-5
num_epochs = 5000
minibatch_size = 20

w_best = None
b_best = None

minimum = 0
no_improve = 0
early_stopping_steps = 29 

def accuracy(X, y):
    correct = 0
    for i in range(len(y)):
        y_predicted = int(np.sign((torch.dot(w, X[i]) - b).detach().numpy()[0]))
        if y_predicted == y[i]: correct += 1
    return float(correct)/len(y)


for epoch in range(num_epochs):
    inds = [i for i in range(len(data_train_tensor))]
    for i in range(len(inds)):
        L = max(0, 1 - labels_train_svm_tensor[inds[i]] * (torch.dot(w, data_train_tensor[inds[i]]) - b))
        if L != 0: # if the loss is zero, Pytorch leaves the variables as a float 0.0, so we can't call backward() on it
            L.backward()
            w.data -= step_size * w.grad.data # step
            b.data -= step_size * b.grad.data # step
            w.grad.data.zero_()
            b.grad.data.zero_()
    
    if epoch % 5 == 4:
        accuracy_val = accuracy(data_val_tensor, labels_val_svm_tensor)
        
        if accuracy_val > minimum:
            minimum = accuracy_val
            w_best = copy.deepcopy(w)
            b_best = copy.deepcopy(b)
            no_improve = 0
        else:
            no_improve += 1

        print("Epoch {}/{}, Accuracy: {:.3f}, Accuracy_Val: {:.3f}".format(epoch+1, num_epochs, 
                        accuracy(data_train_tensor, labels_train_svm_tensor), accuracy(data_val_tensor, labels_val_svm_tensor)))  
    
    # early stopping
    if no_improve > early_stopping_steps:
        break

w = w_best
b = b_best

print("Best Result, Accuracy_Val: {:.3f}".format(accuracy(data_val_tensor, labels_val_svm_tensor)))  
print("Best Result, Accuracy_Test: {:.3f}".format(accuracy(data_test_tensor, labels_test_svm_tensor)))  
    

Epoch 5/5000, Accuracy: 0.843, Accuracy_Val: 0.607
Epoch 10/5000, Accuracy: 0.866, Accuracy_Val: 0.635
Epoch 15/5000, Accuracy: 0.877, Accuracy_Val: 0.627
Epoch 20/5000, Accuracy: 0.877, Accuracy_Val: 0.627
Epoch 25/5000, Accuracy: 0.877, Accuracy_Val: 0.648
Epoch 30/5000, Accuracy: 0.874, Accuracy_Val: 0.627
Epoch 35/5000, Accuracy: 0.871, Accuracy_Val: 0.630
Epoch 40/5000, Accuracy: 0.879, Accuracy_Val: 0.656
Epoch 45/5000, Accuracy: 0.880, Accuracy_Val: 0.656
Epoch 50/5000, Accuracy: 0.872, Accuracy_Val: 0.668
Epoch 55/5000, Accuracy: 0.850, Accuracy_Val: 0.632
Epoch 60/5000, Accuracy: 0.891, Accuracy_Val: 0.710
Epoch 65/5000, Accuracy: 0.897, Accuracy_Val: 0.740
Epoch 70/5000, Accuracy: 0.897, Accuracy_Val: 0.748
Epoch 75/5000, Accuracy: 0.868, Accuracy_Val: 0.689
Epoch 80/5000, Accuracy: 0.887, Accuracy_Val: 0.746
Epoch 85/5000, Accuracy: 0.884, Accuracy_Val: 0.743
Epoch 90/5000, Accuracy: 0.894, Accuracy_Val: 0.771
Epoch 95/5000, Accuracy: 0.879, Accuracy_Val: 0.746
Epoch 100/500